In [5]:
from gremlin_python.driver import client, serializer

account_name = 'gen10-dbc-aug2021-gremlin-api'
db_name = 'library'
collection_name = 'books'
password = 'zBByrlJtXzNXwVpB1IpYcCxTNl0izrwGY4tEQcw436i6V4Dv6VWYrCY7kfmbObfxRtd4ghU2XvjXKSvCNv5Ppg=='

client = client.Client(
        f'wss://{account_name}.gremlin.cosmosdb.azure.com:443/','g', 
        username=f"/dbs/{db_name}/colls/{collection_name}",
    password=password,
        message_serializer=serializer.GraphSONSerializersV2d0()
)

In [6]:
from gremlin_python.driver.protocol import GremlinServerError
import nest_asyncio
# Need to do this to get around thread issues
nest_asyncio.apply() 

cosmosdb_messages = {
    401: 'Invalid credentials',
    409: 'Conflict exception. You\'re probably inserting the same ID again.',
    429: 'Not enough RUs for this query. Try again.'
}

def executeGremlinQuery(gremlinQuery, message=None, params=None):
    try:         
        callback = client.submitAsync(gremlinQuery)
        if callback.result() is not None:
            return callback.result().one()
    except GremlinServerError as ex:
        status=ex.status_attributes['x-ms-status-code']
        print('There was an exception: {0}'.format(status))
        print(cosmosdb_messages[status])
        

In [7]:
executeGremlinQuery("g.V()")

[{'id': 'b818d241-d526-4ed3-97de-2a2ab82016ff',
  'label': 'book',
  'type': 'vertex',
  'properties': {'name': [{'id': 'b818d241-d526-4ed3-97de-2a2ab82016ff|name',
     'value': 'Knowledge is Beautiful'}],
   'author': [{'id': '32cb258b-7dd3-4f51-aad2-d474ecfb982b',
     'value': 'David McCandless'}],
   'msrp': [{'id': '1b67da81-3692-4e6e-bfb7-d9403b26177f', 'value': 30.0}]}},
 {'id': '62b843ad-b6bf-424a-8de7-df49c230351b',
  'label': 'book',
  'type': 'vertex',
  'properties': {'name': [{'id': '62b843ad-b6bf-424a-8de7-df49c230351b|name',
     'value': 'The Visual Miscellaneum'}],
   'author': [{'id': '16e46dbe-fda1-4b9c-b9da-50097ec74758',
     'value': 'David McCandless'}],
   'msrp': [{'id': 'b38f3452-249e-43cb-94b0-19604bc34a8c', 'value': 49.75}]}},
 {'id': '4b5423db-6089-49a2-a1a0-b9453ceb5f44',
  'label': 'book',
  'type': 'vertex',
  'properties': {'name': [{'id': '4b5423db-6089-49a2-a1a0-b9453ceb5f44|name',
     'value': 'Datavision²'}],
   'author': [{'id': '3e22f926-81a6-4c

In [8]:
executeGremlinQuery("g.V().limit(1)")

[{'id': 'b818d241-d526-4ed3-97de-2a2ab82016ff',
  'label': 'book',
  'type': 'vertex',
  'properties': {'name': [{'id': 'b818d241-d526-4ed3-97de-2a2ab82016ff|name',
     'value': 'Knowledge is Beautiful'}],
   'author': [{'id': '32cb258b-7dd3-4f51-aad2-d474ecfb982b',
     'value': 'David McCandless'}],
   'msrp': [{'id': '1b67da81-3692-4e6e-bfb7-d9403b26177f', 'value': 30.0}]}}]

In [9]:
result = executeGremlinQuery("g.E().count()")
print("Count of edges: {0}".format(result))

Count of edges: [2]
